In [1]:
import numpy as np
import time

In [2]:
def create_input(n_input_branch, n_input_trunk, n_batch):

	return np.random.rand(n_batch,n_input_branch)-0.5, np.random.rand(n_batch,n_input_trunk)-0.5


def create_params(n_input, n_neurons):

	j_neurons     = n_input
	A_vec         = []
	b_vec         = []
	for i_neurons in n_neurons:
		A         = np.random.rand(j_neurons,i_neurons)
		b         = np.random.rand(1,i_neurons)
		j_neurons = i_neurons
		A_vec.append(A)
		b_vec.append(b)

	return A_vec, b_vec

In [3]:
def Nonef(x):
    return x


def softplus(x):
    return np.log1p(np.exp(x))


def call_layer(x, A, b, act):
    z = np.dot(x, A) + b
    y = act(z)
    return y


def call_block(x, A_vec, b_vec, acts_vec):
    for i, A in enumerate(A_vec):
        b   = b_vec[i]
        act = acts_vec[i] 
        x   = call_layer(x, A, b, act)
    return x


def call_DeepONet(x_mat, A_mat, b_mat, acts_mat, NReps):
    
    t = time.time()
    
    for k in range(NReps):
        
        y_branch     = call_block(x_mat[0], A_mat[0], b_mat[0], acts_mat[0])
        y_trunk      = call_block(x_mat[1], A_mat[1], b_mat[1], acts_mat[1])

        y            = np.sum(y_branch * y_trunk, axis=1) + b_final
        
    elapsed      = time.time() - t

    return elapsed


def call_flexDeepONet(x_mat, A_mat, b_mat, acts_mat, NReps):
    
    x_branch     = x_mat[0]
    x_trunk      = x_mat[1]
    i            = 1

    t = time.time()
    
#     i           += 1
#     A_vec_rot    = A_mat[i]
#     b_vec_rot    = b_mat[i]
#     acts_rot     = acts_mat[i]
#     y_rot_       = call_block(x_branch,      A_vec_rot,      b_vec_rot,    acts_rot)
#     s_rot        = np.sin(y_rot_[:,0])
#     c_rot        = np.cos(y_rot_[:,0])
#     y_rot        = np.zeros_like(x_trunk)
#     y_rot[:,0]   = x_trunk[:,0] * (c_rot+s_rot)
#     y_rot[:,1]   = x_trunk[:,1] * (c_rot-s_rot)
#     x_trunk      = y_rot
    
#     i           += 1
#     A_vec_shift  = A_mat[i]
#     b_vec_shift  = b_mat[i]
#     acts_shift   = acts_mat[i]
#     y_shift      = call_block(x_branch,    A_vec_shift,    b_vec_shift,   acts_shift)
#     x_trunk     += y_shift 
    
    i           += 1
    A_vec_stretch= A_mat[i]
    b_vec_stretch= b_mat[i]
    acts_stretch = acts_mat[i]
    y_stretch    = call_block(x_branch,  A_vec_stretch,  b_vec_stretch, acts_stretch)
    x_trunk     *= y_stretch 
        
    
    for k in range(NReps):
        y_branch     = call_block(x_branch, A_mat[0], b_mat[0], acts_mat[0])
        y_trunk      = call_block(x_trunk,  A_mat[1], b_mat[1], acts_mat[1])
    
        y            = np.sum(y_trunk * y_branch[:,0:-1], axis=1) + y_branch[:,-1]

    elapsed      = time.time() - t

    return elapsed

In [4]:
# x = np.random.rand((100,10))

# z = np.dot(x, A) + b
# y = act(z)

In [5]:
# ################################################################################
# n_batch           = 10000

# n_input_branch    = 2
# n_input_trunk     = 1

# acts_branch       = [np.tanh, np.tanh, np.tanh, Nonef]
# acts_trunk        = [np.tanh, np.tanh, np.tanh, Nonef]


# n_neurons_branch  = [32, 32, 32, 2] 
# n_neurons_trunk   = [32, 32, 32, 2] 

# NReps             = 2

# Nt                = 100



# ################################################################################
# n_batch          = 10000

# n_input_branch   = 1
# n_input_trunk    = 1

# acts_branch      = [np.tanh, np.tanh, np.tanh, Nonef]
# acts_trunk       = [np.tanh, np.tanh, np.tanh, Nonef]

# n_neurons_branch = [32, 32, 32, 8] 
# n_neurons_trunk  = [32, 32, 32, 8] 

# NReps            = 1

# Nt               = 100



################################################################################
n_batch          = 10000

n_input_branch   = 19
n_input_trunk    = 1

acts_branch      = [np.tanh, np.tanh, np.tanh, np.tanh, np.tanh, Nonef]
acts_trunk       = [np.tanh, np.tanh, np.tanh, np.tanh, np.tanh, Nonef]

n_neurons_branch = [64, 64, 64, 64, 64, 32] 
n_neurons_trunk  = [64, 64, 64, 64, 64, 32] 

NReps            = 19

Nt               = 30



# ################################################################################
# n_batch          = 100*100

# n_input_branch   = 1
# n_input_trunk    = 2

# acts_branch      = [np.tanh, np.tanh, np.tanh, np.tanh, np.tanh, Nonef]
# acts_trunk       = [np.tanh, np.tanh, np.tanh, np.tanh, np.tanh, Nonef]

# n_neurons_branch = [128, 128, 128, 128, 128, 128] 
# n_neurons_trunk  = [128, 128, 128, 128, 128, 128] 

# NReps            = 1

# Nt               = 30

In [6]:
x_mat                      = create_input(n_input_branch, n_input_trunk, n_batch)

A_vec_branch, b_vec_branch = create_params(n_input_branch, n_neurons_branch)
A_vec_trunk,  b_vec_trunk  = create_params(n_input_trunk,  n_neurons_trunk)
b_final                    = np.random.rand()
A_mat                      = [A_vec_branch, A_vec_trunk]
b_mat                      = [b_vec_branch, b_vec_trunk, b_final]

acts_mat                   = [acts_branch, acts_trunk]

In [7]:
tVec1 = []
for it in range(Nt):
    t = call_DeepONet(x_mat, A_mat, b_mat, acts_mat, NReps)
    tVec1.append(t)
    
print(tVec1)

[1.0692050457000732, 1.0610709190368652, 1.046180009841919, 1.0470750331878662, 1.0489699840545654, 1.072772741317749, 1.2200648784637451, 1.110745906829834, 1.0545320510864258, 1.0619797706604004, 1.0605621337890625, 1.0738859176635742, 1.0538489818572998, 1.0504779815673828, 1.0505669116973877, 1.0639119148254395, 1.0489637851715088, 1.0457298755645752, 1.0961809158325195, 1.078359842300415, 1.1614139080047607, 1.074613094329834, 1.056548833847046, 1.0558009147644043, 1.0892012119293213, 1.0842571258544922, 1.0852680206298828, 1.054198980331421, 1.090076208114624, 1.1017792224884033]


In [8]:
tVec1_ = np.array(tVec1)[5::].mean()
print(tVec1_)

1.0798296451568603


In [9]:
# ################################################################################
# n_batch           = 10000

# n_input_branch    = 2
# n_input_trunk     = 1

# acts_branch       = [np.tanh, np.tanh, np.tanh, Nonef]
# acts_trunk        = [np.tanh, np.tanh, np.tanh, Nonef]


# n_neurons_branch  = [32, 32, 32, 3] 
# n_neurons_trunk   = [32, 32, 32, 2] 

# NReps             = 2

# Nt                = 100




# ################################################################################
# n_batch           = 10000

# n_input_branch    = 1
# n_input_trunk     = 1

# acts_branch       = [np.tanh, Nonef]
# acts_stretch      = [Nonef]
# acts_trunk        = [np.tanh]


# n_neurons_branch  = [2, 2] 
# n_neurons_stretch = [1] 
# n_neurons_trunk   = [1] 

# NReps             = 1

# Nt                = 100



################################################################################
n_batch           = 10000

n_input_branch    = 19
n_input_trunk     = 1

acts_branch       = [np.tanh, np.tanh, np.tanh, Nonef]
acts_stretch      = [np.tanh, np.tanh, np.tanh, softplus]
acts_trunk        = [np.tanh, np.tanh, np.tanh, Nonef]


n_neurons_branch  = [16, 16, 16, 9] 
n_neurons_stretch = [16, 16, 16, 1] 
n_neurons_trunk   = [16, 16, 16, 8] 

NReps             = 19

Nt                = 30


# ################################################################################
# n_batch           = 100*100

# n_input_branch    = 1
# n_input_trunk     = 2

# acts_branch       = [np.tanh, np.tanh, Nonef]
# acts_shift        = [np.tanh, np.tanh, np.tanh, Nonef]
# acts_stretch      = [np.tanh, np.tanh, np.tanh, Nonef]
# acts_rot          = [np.tanh, np.tanh, np.tanh, Nonef]
# acts_trunk        = [np.tanh, np.exp,  Nonef]


# n_neurons_branch  = [8,   8,  2   ] 
# n_neurons_shift   = [16, 16, 16, 2] 
# n_neurons_stretch = [16, 16, 16, 1] 
# n_neurons_rot     = [16, 16, 16, 1] 
# n_neurons_trunk   = [4,   1,  1   ] 

# NReps             = 1
# Nt                = 30

In [10]:
x_mat                        = create_input(n_input_branch, n_input_trunk, n_batch)

A_vec_branch,  b_vec_branch  = create_params(n_input_branch,  n_neurons_branch)
A_vec_stretch, b_vec_stretch = create_params(n_input_branch,  n_neurons_stretch)
# A_vec_shift,   b_vec_shift   = create_params(n_input_branch,  n_neurons_shift)
# A_vec_rot,     b_vec_rot     = create_params(n_input_branch,  n_neurons_rot)
A_vec_trunk,   b_vec_trunk   = create_params(n_input_trunk,   n_neurons_trunk)
A_mat                        = [A_vec_branch, A_vec_trunk]
b_mat                        = [b_vec_branch, b_vec_trunk]
acts_mat                     = [acts_branch, acts_trunk]

try:
    A_mat.append(A_vec_rot)
    b_mat.append(b_vec_rot)
    acts_mat.append(acts_rot)
except:
    pass

try:
    A_mat.append(A_vec_shift)
    b_mat.append(b_vec_shift)
    acts_mat.append(acts_shift)
except:
    pass

try:
    A_mat.append(A_vec_stretch)
    b_mat.append(b_vec_stretch)
    acts_mat.append(acts_stretch)
except:
    pass

In [11]:
tVec2 = []
for it in range(Nt):
    t = call_flexDeepONet(x_mat, A_mat, b_mat, acts_mat, NReps)
    tVec2.append(t)
    
print(tVec2)

[0.30910301208496094, 0.266848087310791, 0.24950885772705078, 0.2503199577331543, 0.2466731071472168, 0.24728608131408691, 0.24574518203735352, 0.24628424644470215, 0.24599695205688477, 0.24973106384277344, 0.2492060661315918, 0.25013208389282227, 0.24915194511413574, 0.25057196617126465, 0.24915599822998047, 0.2459111213684082, 0.24931979179382324, 0.24773812294006348, 0.24901986122131348, 0.25264596939086914, 0.24670791625976562, 0.2469639778137207, 0.25000691413879395, 0.24570798873901367, 0.24579405784606934, 0.24744081497192383, 0.24708104133605957, 0.24635601043701172, 0.24851489067077637, 0.2529611587524414]


In [12]:
tVec2_ = np.array(tVec2)[5::].mean()
print(tVec2_)

0.24821724891662597


In [13]:
print(tVec2_/tVec1_)

0.22986704433417268
